In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json
with open('../input/herbarium-2022-fgvc9/train_metadata.json', "r", encoding="ISO-8859-1") as file:
    train = json.load(file)
    
    print(train.keys())    
    train_img = pd.DataFrame(train['images'])
    print ('images')
    print (train_img.head())
    
    train_categories = pd.DataFrame(train['categories'])
    print ('categories')
    print (train_categories.head())
    
    train_genera = pd.DataFrame(train['genera'])
    print ('genera')
    print (train_genera.head())
    
    train_institutions = pd.DataFrame(train['institutions'])
    print ('institutions')
    print (train_institutions.head())
    
    train_ann = pd.DataFrame(train['annotations']).drop(columns='image_id')
    print ('annotations')
    print (train_ann.head())
    
    train_dist = pd.DataFrame(train['distances'])
    print ('distances')
    print (train_dist.head())
    
    #train_df = train_img.merge(train_ann, on='id')
    #train_df.head()

dict_keys(['annotations', 'images', 'categories', 'genera', 'institutions', 'distances', 'license'])
images
     image_id              file_name  license
0  00000__001  000/00/00000__001.jpg        0
1  00000__002  000/00/00000__002.jpg        0
2  00000__003  000/00/00000__003.jpg        0
3  00000__004  000/00/00000__004.jpg        0
4  00000__005  000/00/00000__005.jpg        0
categories
   category_id                                     scientificName    family  \
0            0        Abies amabilis (Douglas ex Loudon) J.Forbes  Pinaceae   
1            1                          Abies balsamea (L.) Mill.  Pinaceae   
2            2                      Abies bracteata (D.Don) Poit.  Pinaceae   
3            3  Abies concolor (Gordon & Glend.) Lindl. ex Hil...  Pinaceae   
4            4                        Abies fraseri (Pursh) Poir.  Pinaceae   

   genus    species                               authors  
0  Abies   amabilis          (Douglas ex Loudon) J.Forbes  
1  Abies  

In [3]:
import tensorflow as tf
import time, json, codecs
from sklearn.model_selection import train_test_split

In [4]:
batchsize=256
learning_rate=0.01
epochs=5
shape = (200, 136, 3)
debug=True
augment_data = False

In [5]:
t_start = time.time()

In [6]:
if not os.path.isdir('weights'):
    os.makedirs('weights')

In [7]:
with codecs.open('../input/herbarium-2022-fgvc9/train_metadata.json','r',encoding='utf-8',errors='ignore') as f:
        train_meta = json.load(f)

In [9]:
train_meta.keys()

dict_keys(['annotations', 'images', 'categories', 'genera', 'institutions', 'distances', 'license'])

In [10]:
train_annotations = pd.DataFrame(train_meta['annotations'])
train_annotations.head()

,genus_id,institution_id,category_id,image_id
0,1,0,0,00000__001
1,1,0,0,00000__002
2,1,0,0,00000__003
3,1,0,0,00000__004
4,1,0,0,00000__005


In [11]:
categories = pd.DataFrame(train_meta['categories'])
categories.head()

,category_id,scientificName,family,genus,species,authors
0,0,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes
1,1,Abies balsamea (L.) Mill.,Pinaceae,Abies,balsamea,(L.) Mill.
2,2,Abies bracteata (D.Don) Poit.,Pinaceae,Abies,bracteata,(D.Don) Poit.
3,3,Abies concolor (Gordon & Glend.) Lindl. ex Hil...,Pinaceae,Abies,concolor,(Gordon & Glend.) Lindl. ex Hildebr.
4,4,Abies fraseri (Pursh) Poir.,Pinaceae,Abies,fraseri,(Pursh) Poir.


In [12]:
train_images = pd.DataFrame(train_meta['images'])
train_images.head()

,image_id,file_name,license
0,00000__001,000/00/00000__001.jpg,0
1,00000__002,000/00/00000__002.jpg,0
2,00000__003,000/00/00000__003.jpg,0
3,00000__004,000/00/00000__004.jpg,0
4,00000__005,000/00/00000__005.jpg,0


In [15]:
Xorig = train_annotations.merge(categories,on='category_id', how="left"
                                     ).merge(train_images, on="image_id", how="outer")

In [16]:
Xorig.head()

,genus_id,institution_id,category_id,image_id,scientificName,family,genus,species,authors,file_name,license
0,1,0,0,00000__001,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__001.jpg,0
1,1,0,0,00000__002,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__002.jpg,0
2,1,0,0,00000__003,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__003.jpg,0
3,1,0,0,00000__004,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__004.jpg,0
4,1,0,0,00000__005,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__005.jpg,0


In [17]:
X = Xorig[['file_name','family','genus','category_id']]
    
name_list = X['family'].unique().tolist()
X.loc[:,'family'] = X['family'].map(lambda x:name_list.index(x))
genus_list = X['genus'].unique().tolist()
X.loc[:,'genus'] = X['genus'].map(lambda x:genus_list.index(x))

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [29]:
print (len(name_list))
#X.head(1000)

272


,file_name,family,genus,category_id
0,000/00/00000__001.jpg,0,0,0
1,000/00/00000__002.jpg,0,0,0
2,000/00/00000__003.jpg,0,0,0
3,000/00/00000__004.jpg,0,0,0
4,000/00/00000__005.jpg,0,0,0
...,...,...,...,...
995,000/18/00018__004.jpg,1,1,18
996,000/18/00018__006.jpg,1,1,18
997,000/18/00018__008.jpg,1,1,18
998,000/18/00018__011.jpg,1,1,18


In [16]:
    with codecs.open('../input/herbarium-2022-fgvc9/test_metadata.json','r',encoding='utf-8',errors='ignore') as f:
        test_meta = json.load(f)
    print (len(test_meta))

210407


In [22]:
debug=True

In [8]:
def load_data():
    
    with codecs.open('../input/herbarium-2022-fgvc9/train_metadata.json','r',encoding='utf-8',errors='ignore') as f:
        train_meta = json.load(f)
    with codecs.open('../input/herbarium-2022-fgvc9/test_metadata.json','r',encoding='utf-8',errors='ignore') as f:
        test_meta = json.load(f)
        
    train_annotations = pd.DataFrame(train_meta['annotations'])
    categories = pd.DataFrame(train_meta['categories'])
    #categories.columns = ['family', 'genus', 'category_id', 'category_name']
    train_images = pd.DataFrame(train_meta['images'])
    #train_images.columns = ['file_name', 'height', 'image_id','license','width']
    print (train_images.columns)
    #X_test = pd.DataFrame(test_meta['images'])
    X_test = pd.DataFrame(test_meta)
    #X_test.columns = ['file_name', 'height', 'image_id','license','width']
    X_test = X_test[['image_id','file_name']]
    
    #regions = pd.DataFrame(train_meta['regions'])
    #regions.columns=['region_id','name']
    
    Xorig = train_annotations.merge(categories,on='category_id', how="left"
                                     ).merge(train_images, on="image_id", how="outer")
    print (Xorig.columns)
    X=Xorig
    #X = Xorig[['file_name','family','genus','category_id']]
    #print (len(X))
    
    #name_list = X['family'].unique().tolist()
    #X.loc[:,'family'] = X['family'].map(lambda x:name_list.index(x))
    #genus_list = X['genus'].unique().tolist()
    #X.loc[:,'genus'] = X['genus'].map(lambda x:genus_list.index(x))
    
    #if debug:
    #    X=X[X['family']>290]
    #return X.astype({'family':'int16','genus':'int16','category_id':'int16'}), X_test
    return X, X_test

In [9]:
X,X_test=load_data()

#nmb_cat = X['category_id'].max()+1
#nmb_gen = X['genus'].max()+1
#nmb_fam = X['family'].max()+1
#X_train, X_dev = train_test_split(X, test_size=0.05, shuffle=True, random_state=13)
#del X



Index(['image_id', 'file_name', 'license'], dtype='object')
Index(['genus_id', 'institution_id', 'category_id', 'image_id',
       'scientificName', 'family', 'genus', 'species', 'authors', 'file_name',
       'license'],
      dtype='object')


In [10]:
X.head()

,genus_id,institution_id,category_id,image_id,scientificName,family,genus,species,authors,file_name,license
0,1,0,0,00000__001,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__001.jpg,0
1,1,0,0,00000__002,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__002.jpg,0
2,1,0,0,00000__003,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__003.jpg,0
3,1,0,0,00000__004,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__004.jpg,0
4,1,0,0,00000__005,Abies amabilis (Douglas ex Loudon) J.Forbes,Pinaceae,Abies,amabilis,(Douglas ex Loudon) J.Forbes,000/00/00000__005.jpg,0


In [11]:
X_test.head()

,image_id,file_name
0,0,000/test-000000.jpg
1,1,000/test-000001.jpg
2,2,000/test-000002.jpg
3,3,000/test-000003.jpg
4,4,000/test-000004.jpg
